---
---
---

# Brain Tumor Segmentation with U-Net 🧩

**Objective**:
Develop and evaluate a resource-efficient U-Net for delineating brain tumor subregions from multi-modal MRI.

---
---
---

---

## 1. Setup 📦

---

In [52]:
# Data Management
import glob
import numpy as np
import pandas as pd

# Data Visualization
from matplotlib.colors import ListedColormap
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import seaborn as sns

# Computational Modules
import math
from scipy.stats import norm, zscore
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, StandardScaler

# Deep Learning Frameworks
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# Utils
from collections import defaultdict
import h5py
from google.colab import drive
from IPython.display import display, HTML, IFrame, Image
import os
from pathlib import Path
import random
import re
from tqdm import tqdm

# Settings
pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment', None)
sns.set_style('darkgrid', {'grid.color':'0.9','xtick.bottom':True,'ytick.left':True})


---

## 2. Data Access 📂

---

In [2]:
# Define project root
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Define file paths relative to drive structure
data_path = '/content/drive/MyDrive/BraTS_Data/BraTS2020_training_data/content/data'
meta_data_path = '/content/drive/MyDrive/BraTS_Data/BraTS2020_training_data/content/data/meta_data.csv'
name_mapping_path = '/content/drive/MyDrive/BraTS_Data/BraTS2020_training_data/content/data/name_mapping.csv'
survival_info_path = '/content/drive/MyDrive/BraTS_Data/BraTS2020_training_data/content/data/survival_info.csv'
slice_paths = sorted(glob.glob(os.path.join(data_path, '*.h5')))  # get all .h5 files

In [5]:
# Load CSVs into local dataframes
meta_data_df = pd.read_csv(meta_data_path)
name_mapping_df = pd.read_csv(name_mapping_path)
survival_info_df = pd.read_csv(survival_info_path)

In [ ]:
### TO BE EXECUTED BEFORE TRAINING FOR FULL DATASET ###
### Copy dataset locally and use instead for faster access ###

# !cp -r /content/drive/MyDrive/BraTS_Data /content/
# data_path = '/content/BraTS_Data/BraTS2020_training_data/content/data'
# slice_paths = sorted(glob.glob(os.path.join(data_path, '*.h5')))

---

## 3. Model Prerequisites ⚙️

---

#### **3a. Split imaging data into training and validation sets.**

For this v1 (baseline model), only a limited set of MRI volumes will be used due to runtime trouble on free-tier Colab. Once an entire workflow is established, a more fleshed-out splitting will take place for future versions.

In [6]:
# Extract random patient/volume IDs for baseline model
random.seed(42)
selected_patients = random.sample(range(0, 368), 4)
print('Randomly selected patient IDs: ', selected_patients)

Randomly selected patient IDs:  [327, 57, 12, 140]


In [7]:
# Convert numeric IDs into corresponding filename prefixes
selected_prefixes = [f'volume_{pid}_' for pid in selected_patients]
selected_prefixes

['volume_327_', 'volume_57_', 'volume_12_', 'volume_140_']

In [8]:
# Collect all slices belonging to selected patients
selected_slice_paths = [
    path for path in slice_paths
    if any(prefix in os.path.basename(path) for prefix in selected_prefixes)
]
print('Total selected slices: ', len(selected_slice_paths))
# selected_slice_paths

Total selected slices:  620


In [9]:
# Copy selected slices to local runtime (for faster I/O during training)
local_data_path = '/content/baseline_data'
os.makedirs(local_data_path, exist_ok=True)
for path in selected_slice_paths:
    filename = os.path.basename(path)
    !cp '{path}' '{local_data_path}/{filename}'

In [12]:
# Rebuild slice list from local directory
local_slice_paths = sorted(glob.glob(os.path.join(local_data_path, '*.h5')))
print('Local slice count: ', len(local_slice_paths))

Local slice count:  620


In [13]:
# Group slices by patient (patient-level split)
patient_to_slices = defaultdict(list)
for path in local_slice_paths:
    filename = os.path.basename(path)
    patient_id = filename.split('_slice')[0]
    patient_to_slices[patient_id].append(path)
print(f'Patients found locally: {list(patient_to_slices.keys())}')

Patients found locally: ['volume_12', 'volume_140', 'volume_327', 'volume_57']


In [14]:
# Seperate train and validation patients
patient_ids = list(patient_to_slices.keys())

train_patients = ['volume_' + str(i) for i in selected_patients[:3]]
val_patients = ['volume_' + str(i) for i in selected_patients[3:]]

print('Train patients:', train_patients)
print('Val patient:', val_patients)

Train patients: ['volume_327', 'volume_57', 'volume_12']
Val patient: ['volume_140']


In [15]:
# Separate train and validation slice paths
train_paths = []
val_paths = []

for pid in train_patients:
    train_paths.extend(patient_to_slices[pid])
for pid in val_patients:
    val_paths.extend(patient_to_slices[pid])

print('Train slices:', len(train_paths))
print('Val slices:', len(val_paths))

Train slices: 465
Val slices: 155


**Patient-Level Subset Selection and Split**

To keep training within runtime limits, a small baseline subset was created by randomly selecting four patients (seed = 42). All slices belonging to these patients were identified and copied locally to the Colab runtime to ensure faster disk access during training.

A deterministic 3:1 patient-level split was then applied. The split is performed at the patient level (not slice level) to prevent data leakage, ensuring that no slices from the same volume appear in both sets.

This setup keeps the experiment lightweight while preserving proper evaluation structure.

#### **3b. Format and standardize slices via PyTorch Dataset.**

In [16]:
class BraTSDataset(Dataset):
    """
    PyTorch Dataset for loading 2D BraTS slices stored as .h5 files.

    Each file contains:
        - image: (H, W, 4)
              MRI modalities in order: [T1, T1Gd, T2, T2-FLAIR]
        - mask:  (H, W, 3)
              Binary tumor subregions: [NEC/NET, ED, ET]

    Output format:
        - image tensor: (4, H, W), float32
        - mask tensor:  (3, H, W), float32
    """

    def __init__(self, slice_paths):
        """
        Args:
            slice_paths (list): List of file paths to .h5 slice files.
        """
        self.slice_paths = slice_paths

    def __len__(self):
        """
        Returns total number of slices and enables use with PyTorch DataLoader.
        """
        return len(self.slice_paths)

    def __getitem__(self, idx):
        """
        Loads and processes a single slice from disk.

        Steps:
            1. Load image + mask from disk (lazy loading)
            2. Convert to float32 (memory + GPU compatible)
            3. Apply per-slice, per-modality z-score normalization
            3. Rearrange to channel-first format (C, H, W) for PyTorch
            4. Convert to torch.Tensor
        """

        file_path = self.slice_paths[idx]

        # Load slice from disk (lazy loading; no full dataset in memory)
        with h5py.File(file_path, 'r') as file:
            image = file['image'][:]   # Shape: (H, W, 4)
            mask  = file['mask'][:]    # Shape: (H, W, 3)

        # Ensure correct dtype for training (convert to float32)
        image = image.astype(np.float32)
        mask  = mask.astype(np.float32)  # Also converted for loss computation; Binary values for each tumor subregion

        # Per-slice & per-modality z-score normalization to standardize intensity dist. independently for each MRI modality channel
        for c in range(image.shape[-1]):
            channel = image[:, :, c]
            mean = channel.mean()
            std = channel.std()
            if std > 0:
                image[:, :, c] = (channel - mean) / std  # Improves optimization stability and handles inter-patient intensity variation
            else:
                image[:, :, c] = channel - mean  # Avoid division by zero if std=0

        # Convert to channel-first format (C, H, W) (required for PyTorch Conv2D layers)
        image = np.transpose(image, (2, 0, 1))  # (4, H, W)
        mask  = np.transpose(mask, (2, 0, 1))   # (3, H, W)

        # Convert numpy arrays to torch tensors
        image_tensor = torch.from_numpy(image)
        mask_tensor  = torch.from_numpy(mask)

        return image_tensor, mask_tensor

In [17]:
# Implement processing steps onto train and validation sets
train_dataset = BraTSDataset(train_paths)
val_dataset   = BraTSDataset(val_paths)

In [18]:
# Inspect example output
sample_img, sample_mask = train_dataset[0]
print('Image shape:', sample_img.shape)   # (4, 240, 240)
print('Mask shape:', sample_mask.shape)   # (3, 240, 240)
print('Image dtype:', sample_img.dtype)
print('Mask dtype:', sample_mask.dtype)

Image shape: torch.Size([4, 240, 240])
Mask shape: torch.Size([3, 240, 240])
Image dtype: torch.float32
Mask dtype: torch.float32


**Dataset Implementation**:

Before training, MRI slices must be properly formatted and standardized to ensure consistent numerical representation, improve training stability, and enable efficient batch loading for the U-Net.

A custom PyTorch `Dataset` class is defined to load individual 2D slices stored as *.h5* files. Each file contains a 4-channel MRI image (T1, T1Gd, T2, T2-FLAIR) and a 3-channel binary segmentation mask representing tumor subregions.

Data is loaded lazily within `__getitem__`, meaning slices are read from disk only when needed. This avoids loading the entire dataset into memory and keeps the pipeline scalable.

Images and masks are converted to *float32* for GPU compatibility and reduced memory usage. Each MRI modality channel is also z-score normalized per slice to standardize intensities and stabilize training. This should improve model convergence and help the network learn more effectively. The arrays are then rearranged to channel-first format **(C, H, W)**, which is required by PyTorch convolutional layers. A sample output was generated to inspect it for expected format.

This design separates data handling from modeling logic, making the pipeline modular, extensible, and ready for the U-Net baseline.

#### **3c. Create DataLoaders for batching and training.**

The next step is to set up data generators to produce iterable batches for training and validation.

In [19]:
# Configure batch size based on v1 constrainsts (GPU + slice size)
batch_size = 4

In [20]:
# Create DataLoaders to handles batching, shuffling, parallel loading, and GPU memory pinning
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,            # shuffle training slices each epoch (ensures SGD sees slices in different order each epoch)
    num_workers=2,           # parallel data loading; adjust based on Colab memory / CPU
    pin_memory=True          # speeds up transfers to GPU
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,            # maintain deterministic order (i.e., no shuffle) for validation
    num_workers=2,
    pin_memory=True
)

In [21]:
sample_images, sample_masks = next(iter(train_loader))
print('Batch image shape:', sample_images.shape)  # (B, 4, H, W)
print('Batch mask shape: ', sample_masks.shape)   # (B, 3, H, W)

Batch image shape: torch.Size([4, 4, 240, 240])
Batch mask shape:  torch.Size([4, 3, 240, 240])


**DataLoader Implementation**:

With the Dataset class defined, we now set up PyTorch `DataLoader` to efficiently batch and serve slices during training and validation.

- **Training DataLoader**: shuffles slices each epoch to provide stochasticity for gradient descent, uses multiple workers for parallel loading, and pins memory for faster GPU transfer.
- **Validation DataLoader**: does not shuffle to maintain deterministic evaluation order but still benefits from batching and parallel loading.

Inspecting a sample batch image and mask shape confirms that each batch has the expected shape: images **(B, 4, H, W)** and masks **(B, 3, H, W)**. This ensures that data is correctly formatted, normalized, and ready for input into a segmentation model.

---

## 4. Model Architecture 🏗️

---

Now that the data is preprocessed and loaded in batches, the next step is to define the neural network. A **U-Net** will be used as it is specifically designed for image segmentation.

- U-Net takes an input image (here, 4 MRI channels) and outputs a segmentation mask (here, 3 channels for necrotic core, edema, enhancing tumor).
- It has a **downsampling path** that captures context and an **upsampling path** that recovers spatial details.
- The network will learn, for each pixel, whether it belongs to background or one of the tumor sub-regions.

This setup allows the model to predict multi-class masks directly from MRI slices.

#### **4a. Define a baseline U-Net for brain tumor segmentation.**

In [22]:
class LeanUNet(nn.Module):
    """
    U-Net (v1) for multi-class 2D brain tumor segmentation.

    Input: 4-channel MRI slice (T1, T1Gd, T2, T2-FLAIR)
    Output: 3-channel segmentation mask (Necrotic Core, Edema, Enhancing Tumor)
    """


    def __init__(self, in_channels=4, out_channels=3, init_features=16):
        super(LeanUNet, self).__init__()
        features = init_features

        ### Encoder (downsampling path)
        self.encoder1 = self._conv_block(in_channels, features)
        self.pool1    = nn.MaxPool2d(2)

        self.encoder2 = self._conv_block(features, features*2)
        self.pool2    = nn.MaxPool2d(2)

        self.encoder3 = self._conv_block(features*2, features*4)
        self.pool3    = nn.MaxPool2d(2)

        ### Bottleneck
        self.bottleneck = self._conv_block(features*4, features*8)

        #### Decoder (upsampling path)
        self.upconv3 = nn.ConvTranspose2d(features*8, features*4, kernel_size=2, stride=2)
        self.decoder3 = self._conv_block(features*8, features*4)

        self.upconv2 = nn.ConvTranspose2d(features*4, features*2, kernel_size=2, stride=2)
        self.decoder2 = self._conv_block(features*4, features*2)

        self.upconv1 = nn.ConvTranspose2d(features*2, features, kernel_size=2, stride=2)
        self.decoder1 = self._conv_block(features*2, features)

        ### Output layer
        self.conv_final = nn.Conv2d(features, out_channels, kernel_size=1)


    def forward(self, x):
        ### Encoder forward pass
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(self.pool1(enc1))
        enc3 = self.encoder3(self.pool2(enc2))

        ### Bottleneck
        bottleneck = self.bottleneck(self.pool3(enc3))

        ### Decoder forward pass with skip connections
        dec3 = self.upconv3(bottleneck)  # upsample
        dec3 = torch.cat((dec3, enc3), dim=1)  # concatenate skip connection from encoder
        dec3 = self.decoder3(dec3)

        dec2 = self.upconv2(dec3)
        dec2 = torch.cat((dec2, enc2), dim=1)
        dec2 = self.decoder2(dec2)

        dec1 = self.upconv1(dec2)
        dec1 = torch.cat((dec1, enc1), dim=1)
        dec1 = self.decoder1(dec1)

        ### Output
        return self.conv_final(dec1)


    ### 2-conv block
    def _conv_block(self, in_channels, out_channels):
        """
        Standard double-convolution block:
            Conv2d -> ReLU -> Conv2d -> ReLU
        """
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

In [23]:
# Define computation mode
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device: ', device)

# Instantiate model
model = LeanUNet().to(device)
# print(model)

# Check output shape for an example batch (will be consolidated later as unit test modules)
ex_input = torch.randn(2, 4, 240, 240).to(device)
ex_output = model(ex_input)
print('Example output shape:', ex_output.shape)  # Expected: (2, 3, 240, 240)

Device:  cuda
Example output shape: torch.Size([2, 3, 240, 240])


**U-Net Architecture**:

- **DoubleConv block**: Two consecutive convolution layers with ReLU activation capture local features efficiently.  
- **Downsampling path**: `DoubleConv + MaxPool` layers reduce spatial dimensions while increasing feature depth, learning contextual information.  
- **Bottleneck**: The deepest layer with the highest feature dimension, capturing the most abstract representation of the input.  
- **Upsampling path**: Transposed convolutions increase spatial resolution, and skip connections concatenate features from the corresponding downsampling layers to recover fine details.  
- **Output layer**: A 1×1 convolution maps features to `out_ch` channels (3 tumor classes).

**Input / Output**:

- **Input**: (C=4, H=240, W=240) MRI slice with four modalities (T1, T1Gd, T2, T2-FLAIR).  
- **Output**: (C=3, H=240, W=240) multi-class segmentation mask, one channel per tumor sub-region.

This design allows the model to predict spatially precise tumor regions while leveraging both global context and local details.


#### **4b. Specify loss function for multi-class segmentation.**


In [24]:
class DiceLoss(nn.Module):
    """
    Computes Dice Loss for multi-channel segmentation.
    Handles multi-label targets with one channel per class.
    """

    def __init__(self, smooth=1e-6):
        super(DiceLoss, self).__init__()
        self.smooth = smooth

    def forward(self, preds, targets):
        # Predictions and Targets: (B, C, H, W)
        preds = torch.sigmoid(preds)  # Ensure outputs in [0,1]
        intersection = (preds * targets).sum(dim=(2,3))
        union = preds.sum(dim=(2,3)) + targets.sum(dim=(2,3))
        dice = (2. * intersection + self.smooth) / (union + self.smooth)
        return 1 - dice.mean()  # 1 - mean Dice across batch and channels

In [25]:
# Instantiate loss function
criterion = DiceLoss()

# Inspect with sample tensors (will be consolidated later as unit test modules)
ex_preds = torch.randn(2, 3, 240, 240).to(device)
ex_targets = torch.randint(0, 2, (2, 3, 240, 240)).float().to(device)
loss = criterion(ex_preds, ex_targets)
print('Sample Dice loss: ', loss.item())

Sample Dice loss:  0.49946802854537964


**Loss Function for Multi-Class Segmentation**:

Brain tumor segmentation exhibits extreme class imbalance, with most pixels belonging to healthy tissue / background. Dice Loss is well-suited for this scenario, as it directly measures the overlap between predicted and ground-truth masks.

- **Multi-channel Dice Loss**: Computed independently per tumor sub-region, then averaged.  
- **Sigmoid activation**: Converts raw logits to probabilities in [0,1] for each channel.  
- **Smooth term**: Prevents division by zero when a class is absent in a slice.

This ensures the model learns to capture small, clinically relevant tumor regions rather than being dominated by background pixels. Combining Dice Loss with standard binary cross-entropy can also stabilize training for future versions.

#### **4c. Configure optimizer and learning rate scheduler.**


In [26]:
# Configure Adam optimizer (adaptive learning rates, fast convergence)
learning_rate = 1e-3
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [27]:
# Set up LR scheduler to reduce LR if validation Dice plateaus (help convergences)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='max',           # maximize validation Dice
    factor=0.5,           # reduce LR by half
    patience=3,           # wait 3 epochs without improvement
)

In [28]:
print('Optimizer parameters count: ', sum(p.numel() for p in model.parameters()))

Optimizer parameters count:  482915


**Optimizer and Learning Rate Scheduler**:

- **Optimizer**: Adam is chosen for its adaptive learning rate capabilities and fast convergence, which is suitable for small baseline datasets.  
- **Learning rate**: Initialized at 1e-3, balancing stability and speed of convergence.  
- **Scheduler**: `ReduceLROnPlateau` monitors validation Dice; if no improvement occurs for 3 epochs, the learning rate is halved to encourage further optimization.

This setup provides a flexible and efficient optimization loop for training the baseline U-Net without manual LR tuning.

#### **4d. Define validation Dice evaluation.**


In [29]:
def compute_dice(preds, targets, threshold=0.5, smooth=1e-6):
    """
    Compute per-channel and mean Dice coefficient for multi-label masks.

    Args:
        preds (torch.Tensor): Model outputs (B, C, H, W) raw logits.
        targets (torch.Tensor): Ground-truth masks (B, C, H, W) in {0,1}.
        threshold (float): Sigmoid threshold to binarize predictions.
        smooth (float): Small value to avoid division by zero.

    Returns:
        dice_per_channel (list of float): Dice for each mask channel.
        mean_dice (float): Average Dice across channels.
    """

    # Apply sigmoid and binarize
    preds = torch.sigmoid(preds) > threshold

    # Flatten spatial dimensions
    preds_flat = preds.view(preds.size(0), preds.size(1), -1).float()
    targets_flat = targets.view(targets.size(0), targets.size(1), -1).float()

    # Compute intersection and union per channel
    intersection = (preds_flat * targets_flat).sum(dim=2)
    union = preds_flat.sum(dim=2) + targets_flat.sum(dim=2)

    # Dice per channel for each sample
    dice = (2 * intersection + smooth) / (union + smooth)

    # Average over batch
    dice_per_channel = dice.mean(dim=0).cpu().tolist()
    mean_dice = dice.mean().item()

    return dice_per_channel, mean_dice

**Validation Dice Evaluation**:

This function computes Dice scores for multi-label segmentation:

- **Per-channel Dice**: Calculates Dice for each tumor sub-region independently.  
- **Mean Dice**: Average across all channels to summarize overall performance.  
- **Binarization**: Predictions are converted from raw logits to 0/1 using a sigmoid threshold (default 0.5).  
- **Batch-wise averaging**: Dice is first computed per slice, then averaged over the batch.

This helper allows efficient evaluation of model performance on the validation set at the end of each epoch or after training without adding significant compute overhead.

#### **4e. Set up remaining training parameters.**


In [30]:
# Training parameters
num_epochs = 50
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Move model to device (GPU or CPU)
model.to(device)

print(f'Training is set up to run on {device} for {num_epochs} epochs with batch size {batch_size}.')

Training is set up to run on cuda for 8 epochs with batch size 4.


The baseline U-Net, loss function, optimizer, scheduler, and training parameters are now fully defined. The model is moved to GPU if available for faster computation. **DiceLoss** handles each mask channel as an independent binary label, making it ideal for multi-label segmentation. The **Adam optimizer** provides adaptive learning rates for efficient convergence. `num_epochs` sets how many times the dataset is seen during training, and `batch_size` controls memory usage and gradient stability. With all components in place, the model is ready for training in the next section.

---

## 5. Model Training 🤖

---

In [31]:
# Set up path for checkpoint saves
drive_models_path = '/content/drive/MyDrive/modelsv1'
os.makedirs(drive_models_path, exist_ok=True)

print('Checkpoints will be saved to: ', drive_models_path)

Checkpoints will be saved to:  /content/drive/MyDrive/modelsv1



For each batch, the process is as follows:

Feed MRI slices into the model --> Compute predictions --> Calculate loss --> Backpropagate --> Update weights.  The loss will be tracked over epochs to see if the model is learning.

In [33]:
for epoch in range(1, num_epochs):  # num_epochs+1*


    # -------------------------------
    # Training Phase
    # -------------------------------
    model.train()       # Enable gradients and training behaviors
    train_loss = 0.0    # Accumulate loss over epoch

    for images, masks in tqdm(train_loader, desc=f'Epoch {epoch} [Train]'):
        images = images.to(device)
        masks = masks.to(device)

        optimizer.zero_grad()            # Reset gradients
        outputs = model(images)          # Forward pass
        loss = criterion(outputs, masks) # Dice loss
        loss.backward()                  # Backpropagate
        optimizer.step()                 # Update weights

        train_loss += loss.item() * images.size(0)

    train_loss /= len(train_loader.dataset)  # Average over all slices


    # -------------------------
    # Checkpoint Save (in case of Colab timeouts)
    # -------------------------
    checkpoint_path = os.path.join(drive_models_path, f'v1_epoch_{epoch}.pt')

    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': train_loss,
    }, checkpoint_path)

    print(f'Checkpoint saved: {checkpoint_path}')


    # -------------------------------
    # Validation Phase
    # -------------------------------
    # Set model to evaluation mode (disables training behaviors, dropout, batchnorm updates)
    model.eval()
    val_loss = 0.0
    dice_per_channel_accum = []

    # Disable gradient computation for validation
    with torch.no_grad():
        for images, masks in tqdm(val_loader, desc=f'Epoch {epoch} [Val]'):
            images = images.to(device)
            masks = masks.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, masks)
            val_loss += loss.item() * images.size(0)

            # Compute Dice per channel for this batch
            dice_ch, mean_dice = compute_dice(outputs, masks)
            dice_per_channel_accum.append(dice_ch)

    # Average validation metrics across all batches
    val_loss /= len(val_loader.dataset)
    dice_per_channel_avg = [sum(col)/len(col) for col in zip(*dice_per_channel_accum)]
    mean_dice_avg = sum(dice_per_channel_avg)/len(dice_per_channel_avg)

    # -------------------------------
    # Scheduler Step
    # -------------------------------
    scheduler.step(mean_dice_avg)  # Adjust LR based on validation Dice

    # -------------------------------
    # Epoch Summary
    # -------------------------------

    print(f'Epoch [{epoch}/{num_epochs}] | '
          f'Train Loss: {train_loss:.4f} | '
          f'Val Loss: {val_loss:.4f} | '
          f'Mean Dice: {mean_dice_avg:.4f} | '
          f'Dice per channel: {dice_per_channel_avg}')

Epoch 1 [Train]: 100%|██████████| 117/117 [00:06<00:00, 17.76it/s]


Checkpoint saved: /content/drive/MyDrive/modelsv1/v1_epoch_1.pt


Epoch 1 [Val]: 100%|██████████| 39/39 [00:01<00:00, 24.17it/s]


Epoch [1/8] | Train Loss: 0.8734 | Val Loss: 0.8296 | Mean Dice: 0.3083 | Dice per channel: [0.21273225898345652, 0.316785276442985, 0.39523367972008566]


Epoch 2 [Train]: 100%|██████████| 117/117 [00:05<00:00, 22.96it/s]


...

Epoch [48/8] | Train Loss: 0.7869 | Val Loss: 0.8274 | Mean Dice: 0.5325 | Dice per channel: [0.48098334522082153, 0.4818520928453866, 0.6346212610801387]


Epoch 49 [Train]: 100%|██████████| 117/117 [00:05<00:00, 20.45it/s]


Checkpoint saved: /content/drive/MyDrive/modelsv1/v1_epoch_49.pt


Epoch 49 [Val]: 100%|██████████| 39/39 [00:01<00:00, 25.47it/s]

Epoch [49/8] | Train Loss: 0.7878 | Val Loss: 0.8359 | Mean Dice: 0.4966 | Dice per channel: [0.4283741005923439, 0.4566021029487874, 0.6048947491646743]


This loop iterates over epochs, updating the model on training slices and evaluating its performance on the validation set:

- **Training phase**: Computes Dice loss, backpropagates, and updates weights via Adam optimizer.  
- **Validation phase**: Disables gradients and computes both loss and Dice metrics for each tumor sub-region using the evaluation helper function from the previous section.  
- **Per-channel Dice**: Monitors performance on necrotic core, edema, and enhancing tumor independently.  
- **Mean Dice**: Average of all channels provides a single performance summary.  
- **Scheduler step**: Optionally reduces learning rate if validation Dice plateaus.  
- **Epoch summary**: Prints training loss, validation loss, mean Dice, and per-channel Dice for monitoring progress.

This setup ensures the baseline model is trained efficiently while keeping track of clinically relevant metrics despite extreme class imbalance.

---

## 6. Model Evaluation 📋

---

In [35]:
# Set model to evaluation mode
model.eval()

final_val_loss = 0.0                     # Accumulate total validation loss
dice_per_channel_accum = []              # Store per-batch Dice scores

with torch.no_grad():

    # Iterate through validation DataLoader
    for images, masks in tqdm(val_loader, desc='Final Validation'):

        # Move data to same device as model (GPU if available)
        images = images.to(device)
        masks = masks.to(device)

        # Forward pass only (no backpropagation)
        outputs = model(images)

        # Compute validation loss using DiceLoss
        loss = criterion(outputs, masks)

        # Accumulate loss weighted by batch size
        final_val_loss += loss.item() * images.size(0)

        # Compute Dice per channel for this batch
        # compute_dice applies sigmoid + threshold internally
        dice_ch, _ = compute_dice(outputs, masks)

        # Store per-channel Dice values
        dice_per_channel_accum.append(dice_ch)

# Average validation loss over total number of slices
final_val_loss /= len(val_loader.dataset)

# Average Dice per channel across all batches
dice_per_channel_avg = [
    sum(col) / len(col) for col in zip(*dice_per_channel_accum)
]

# Compute overall mean Dice across channels
mean_dice_avg = sum(dice_per_channel_avg) / len(dice_per_channel_avg)

# Print final evaluation metrics
print('----- Final Validation Results -----')
print(f'Validation Loss: {final_val_loss:.4f}')
print(f'Mean Dice: {mean_dice_avg:.4f}')
print(f'Dice per channel: {dice_per_channel_avg}')

Final Validation: 100%|██████████| 39/39 [00:01<00:00, 25.07it/s]

----- Final Validation Results -----
Validation Loss: 0.8359
Mean Dice: 0.4966
Dice per channel: [0.4283741005923439, 0.4566021029487874, 0.6048947491646743]


**Final Validation Metrics**:

After ~50 epochs of training, the model is evaluated on the held-out validation patient. The model achieves a validation loss of 0.8359 and a mean Dice score of 0.4966 on the held-out validation set.

Dice performance varies across channels:

- **NEC/NET**: 0.4284
- **ED**: 0.4566
- **ET**: 0.6049

This indicates the model is learning meaningful spatial structure, with the strongest performance on the third class. However, overall Dice remains around 0.5, suggesting segmentation quality is moderate and definitely can be improved in future versions. As a baseline U-Net trained on heavily downscaled slices, this serves as a functional reference point.

In [69]:
# Select a specific validation slice
slice_index = 33

image, mask = val_dataset[slice_index]

# Add batch dimension (model expects B,C,H,W)
image = image.unsqueeze(0).to(device)
mask = mask.unsqueeze(0).to(device)

# Set model to eval mode
model.eval()

with torch.no_grad():
    output = model(image)
    probs = torch.sigmoid(output)
    pred = (probs > 0.5).float()

# Move everything to CPU for plotting
image = image.cpu()
mask = mask.cpu()
pred = pred.cpu()

# Remove batch dimension for plotting
image = image[0]
mask = mask[0]
pred = pred[0]

# Set up figure
fig, axes = plt.subplots(2, 3, figsize=(10, 10))

# Iterable colors and plot titles
channel_colors = ['maroon', 'yellowgreen', 'red']
actual_titles = ['Annotated NEC/NET', 'Annotated ED', 'Annotated ET']
pred_titles = ['Predicted NEC/NET', 'Predicted ED', 'Predicted ET']

# Ground Truth
for c in range(3):
    annotated_cmap = ListedColormap(['black', channel_colors[c]])
    axes[0, c].imshow(mask[c], cmap=annotated_cmap, vmin=0, vmax=1)
    axes[0, c].set_title(actual_titles[c])
    axes[0, c].axis('off')

# Predictions
for c in range(3):
    pred_cmap = ListedColormap(['black', channel_colors[c]])
    axes[1, c].imshow(pred[c], cmap=pred_cmap, vmin=0, vmax=1)
    axes[1, c].set_title(pred_titles[c])
    axes[1, c].axis('off')

plt.tight_layout()
plt.show()

---

## Conclusions

---

This baseline version implemented a multi-class brain tumor segmentation pipeline trained with Dice loss, achieving a mean validation Dice score of 0.4966. Performance varied across tumor subregions, with enhancing tumor (ET) segmented more reliably than necrotic and edema regions—an expected pattern given class imbalance and irregular boundaries.

Visual evaluation revealed extensive slice-to-slice variability. Some predictions exceeded what the average Dice score suggests, while others fell short, reinforcing the need to assess both quantitative and qualitative results in medical imaging tasks.

Several improvements could strengthen this baseline for future versions:

- **Class imbalance handling:** weighted Dice, hybrid Dice + BCE, focal loss, or foreground-aware sampling
- **3D context modeling:** 2.5D inputs or full 3D U-Net architectures
- **Stronger augmentation:** elastic deformations, affine transforms, intensity shifts
- **Architectural upgrades:** attention mechanisms, residual backbones, pretrained encoders
- **Post-processing:** connected-component filtering to reduce false positives
- **Cross-validation:** more reliable performance estimation across folds

Overall, this notebook establishes a clean, reproducible baseline with a solid training and evaluation pipeline. While performance is modest, the structure is robust and ready for improved iterations.